In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import pandas as pd

from opt_weights import *

In [3]:
start_date = datetime.date(2010,1,1)
end_date = datetime.date(2012,5,1)

In [4]:
import pandas.io.data as web
px = web.DataReader(["^GSPC", "^DJI", "^IXIC", "^FCHI", "^FTSE", "^N225"] , 'yahoo', start_date, end_date).Close
ret = px.div(px.shift()).dropna()

/Users/ryanchouck/dev/tf/venv/lib/python2.7/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [9]:
#grid = cum_prod_grid(ret)
grid[grid.norm_type==2].pivot(index='alpha', columns='look_ahead_per', values='cum_ret')

look_ahead_per,1,6,11,16,21,26
alpha,,,,,,
0.000045,1.074351,1.040442,1.102394,1.121914,1.098612,1.093912
0.000172,1.122300,1.066825,1.125839,1.128785,1.111956,1.101439
0.000653,1.126039,1.101111,1.129676,1.140003,1.125449,1.120464
0.002479,1.094476,1.094307,1.115230,1.114943,1.116591,1.115310
0.009404,1.050668,1.061086,1.080516,1.084019,1.085137,1.085146
0.035674,1.036760,1.042213,1.049367,1.051598,1.051837,1.052308
0.135335,1.032185,1.033966,1.036015,1.036746,1.036821,1.036982
0.513417,1.030868,1.031373,1.031929,1.032138,1.032164,1.032213
1.947734,1.030512,1.030648,1.030797,1.030853,1.030860,1.030874


In [12]:
opt_weights_func = lambda x: calc_opt_weights(x, alpha=0.000653, norm_type=2)
weights = rolling_fit_opt_weights(ret, opt_weights_func, 20)

In [6]:
# make sure this doesn't peek ahead
def build_returns_vector(df):
    if df.isnull().values.any():
        raise ValueError("dataframe cannot contain nans")
    df = df.shift()
    pers = (1, 5, 10, 20, 30, 60)
    pn = pd.Panel({i: df.div(df.shift(i)) for i in pers})
    return pn.ix[:,61:,:]

pn = build_returns_vector(px.fillna(method='ffill').dropna())

In [47]:
# make sure Xs and ys are same size
def build_X_y(ret_vecs, weights):
    dates = pn.major_axis
    X y, = [], []
    for ind, date in enumerate(dates):
        try:
            X_row = pn.ix[:,date,:].values.flatten()
            y_row = weights.ix[date].values
            X.append(X_row)
            y.append(y_row)
        except:
            pass
    X = np.array(X)
    y = np.array(y)
    return X, y

Xs, ys = build_X_y(pn, weights)

In [63]:
def get_batch(Xs, ys, batch_size):
    inds = np.random.choice(Xs.shape[0], batch_size, replace=True)
    return Xs[inds,:], ys[inds,:]

In [46]:
import tensorflow as tf

linear model w softmax

In [53]:
x = tf.placeholder(tf.float32, [None, Xs.shape[1]])
y_ = tf.placeholder(tf.float32, [None, ys.shape[1]])
W = tf.Variable(tf.zeros([Xs.shape[1], ys.shape[1]]))
b = tf.Variable(tf.zeros([ys.shape[1]]))
y = tf.nn.softmax(tf.matmul(x , W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [73]:
for i in range(1000):
    batch_Xs, batch_ys = get_batch(Xs, ys, 100)
    sess.run(train_step, feed_dict={x: batch_Xs, y_: batch_ys})

In [74]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [75]:
acc = sess.run(correct_prediction, feed_dict={x: Xs, y_: ys})
(sum(acc) * 1.) / len(acc)

0.38018433179723504

two hidden layer softmax

In [81]:
import math

X_width = Xs.shape[1]
y_width = ys.shape[1]

x = tf.placeholder(tf.float32, [None, Xs.shape[1]])
y_ = tf.placeholder(tf.float32, [None, ys.shape[1]])

hidden1_units = 25
hidden2_units = 10

# Hidden 1
with tf.name_scope('hidden1'):
    weights = tf.Variable(tf.truncated_normal([X_width, hidden1_units], 
                                              stddev=1.0 / math.sqrt(float(X_width))),
                          name='weights')
    biases = tf.Variable(tf.zeros([hidden1_units]),name='biases')
    hidden1 = tf.nn.relu(tf.matmul(x, weights) + biases)
# Hidden 2
with tf.name_scope('hidden2'):
    weights = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units], 
                                              stddev=1.0 / math.sqrt(float(hidden1_units))),
                          name='weights')
    biases = tf.Variable(tf.zeros([hidden2_units]),name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
# Linear
with tf.name_scope('softmax_linear'):
    weights = tf.Variable(tf.truncated_normal([hidden2_units, y_width],
                                               stddev=1.0 / math.sqrt(float(hidden2_units))),
                          name='weights')
    biases = tf.Variable(tf.zeros([y_width]),name='biases')
    logits = tf.matmul(hidden2, weights) + biases
    y = tf.nn.softmax(logits)

In [82]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [86]:
for i in range(1000):
    batch_Xs, batch_ys = get_batch(Xs, ys, 100)
    sess.run(train_step, feed_dict={x: batch_Xs, y_: batch_ys})

In [87]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [88]:
acc = sess.run(correct_prediction, feed_dict={x: Xs, y_: ys})
(sum(acc) * 1.) / len(acc)

0.37327188940092165